In [1]:
import torch 
import math 
import numpy as np
import copy
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import ast 
from numpy import load
import torch.nn as nn
import random
import time
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
import collections
#import helper
import numpy as np
#import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
import spacy
import torchtext
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

In [4]:
import torch.nn as nn

class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [5]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 120):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], \
        requires_grad=False).cuda(1)
        return x

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)
        # calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [7]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    
        
    if mask is not None:
        #print(mask.shape,"mask")
        mask = mask.unsqueeze(1)
        #print(scores.shape, "scores")
        scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

In [8]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [9]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [10]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x
    
# build a decoder layer with two multi-head attention layers and
# one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)
        
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)
        
        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).cuda(1)

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x
        # We can then build a convenient cloning function that can generate multiple layers:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [11]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [12]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output

In [20]:
import pandas as pd 

#test_data=open("/media/data6TB/deepak/NalinIdea/deu-eng/deu.txt")
#daat=[]
#for l in test_data.readlines():
#    daat.append(l.strip().split("\t")[:2])

#test_data.close()

#rest_df = pd.DataFrame(daat[:100000], columns = ['English', 'German'])
#rest_df.to_csv("/media/data6TB/deepak/NalinIdea/deu-eng/new_train_1.csv", index=False)
#test_df = pd.DataFrame(daat[100000:], columns = ['English', 'German'])

import spacy
import torchtext
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

en = spacy.load('en')
de = spacy.load('de')
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
def tokenize_de(sentence):
    return [tok.text for tok in de.tokenizer(sentence)]
EN_TEXT = Field(tokenize=tokenize_en, init_token = "<sos>", eos_token = "<eos>")
DE_TEXT = Field(tokenize=tokenize_de, init_token = "<sos>", eos_token = "<eos>")

from sklearn.model_selection import train_test_split

# create train and validation set 
#train, val = train_test_split(test_df, test_size=0.01)
#train.to_csv("/media/data6TB/deepak/NalinIdea/deu-eng/new_train_2.csv", index=False)
#val.to_csv("/media/data6TB/deepak/NalinIdea/deu-eng/new_val.csv", index=False)

from torchtext.legacy import data
data_fields = [('English', EN_TEXT), ('German', DE_TEXT)]
train,val = data.TabularDataset.splits(path='/media/data6TB/deepak/NalinIdea/deu-eng/', train='new_train_2.csv', validation='new_val.csv', format='csv', fields=data_fields)

DE_TEXT.build_vocab(train, val)
EN_TEXT.build_vocab(train, val)

seq_len=0
for x in train.German:
    if len(x)>seq_len:
        seq_len = len(x)
print(seq_len)
i=0
for x in train.English:
    i=i+1
    if len(x)==0:
        print(i)
i=0
for x in train.German:
    i=i+1
    if len(x)==0:
        print(i)


global max_src_in_batch, max_tgt_in_batch
def batch_size_fn(new, count, sofar):
    "Keep augmenting batch and calculate total number of tokens + padding."
    global max_src_in_batch, max_tgt_in_batch
    if count == 1:
        max_src_in_batch = 0
        max_tgt_in_batch = 0
    max_src_in_batch = max(max_src_in_batch,  len(new.English))
    max_tgt_in_batch = max(max_tgt_in_batch,  len(new.German))
    src_elements = count * max_src_in_batch
    tgt_elements = count * max_tgt_in_batch
    return max(src_elements, tgt_elements)
class MyIterator(data.Iterator):
    def create_batches(self):
        if self.train:
            def pool(d, random_shuffler):
                for p in data.batch(d, self.batch_size * 100):
                    p_batch = data.batch(
                        sorted(p, key=self.sort_key),
                        self.batch_size, self.batch_size_fn)
                    for b in random_shuffler(list(p_batch)):
                        yield b
            self.batches = pool(self.data(), self.random_shuffler)
            
        else:
            self.batches = []
            for b in data.batch(self.data(), self.batch_size,
                                          self.batch_size_fn):
                self.batches.append(sorted(b, key=self.sort_key))

train_iter = MyIterator(train, batch_size=64, device=1,
                        repeat=False, sort_key= lambda x:
                        (len(x.English), len(x.German)),
                        batch_size_fn=batch_size_fn, train=True,
                        shuffle=True)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


85


In [21]:
batch = next(iter(train_iter))
input_seq = batch.English.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']
# creates mask with 0s wherever there is padding in the input
input_msk = (input_seq != input_pad).unsqueeze(1)

from torch.autograd import Variable
import torch
import numpy as np
# create mask as before
target_seq = batch.German.transpose(0,1)
target_pad = DE_TEXT.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)
size = target_seq.size(1) 
# get seq_len for matrix
nopeak_mask = np.triu(np.ones((1, size, size)),k=1).astype('uint8')
nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)
target_msk = target_msk & nopeak_mask

input_msk = input_msk.cuda(1)
target_msk = target_msk.cuda(1)

In [22]:

def nopeak_mask(size,cuda_enabled):
    np_mask = np.triu(np.ones((1, size, size)),
    k=1).astype('uint8')
    np_mask =  torch.autograd.Variable(torch.from_numpy(np_mask) == 0)

    if cuda_enabled:
        np_mask = np_mask.cuda(1)
    return np_mask

def create_masks(src, trg):
    src_mask = (src != 0).unsqueeze(-2)
    if trg is not None:
        trg_mask = (trg != 0).unsqueeze(-2)
        size = trg.size(1) # get seq_len for matrix
        # print("Sequence lenght in mask ",size)
        np_mask = nopeak_mask(size,True)
        # print(np_mask.shape,trg_mask.shape)
        if trg.is_cuda:
            np_mask.cuda(1)
        trg_mask = trg_mask & np_mask
    else:
        trg_mask = None
    return src_mask, trg_mask

In [23]:
d_model = 80
heads = 4
N = 3
src_vocab = len(EN_TEXT.vocab)
trg_vocab = len(DE_TEXT.vocab)

model = Transformer(src_vocab, trg_vocab, d_model, N, heads)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
        
optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [67]:
trg_vocab

33457

In [24]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
embed_parm = sum([np.prod(p.size()) for p in model.decoder.embed.parameters()])  + sum([np.prod(p.size()) for p in model.encoder.embed.parameters()])
ffn_parm = sum([np.prod(p.size()) for p in model.out.parameters()])
params,params-embed_parm-ffn_parm

(8884945, 2214848)

In [27]:
def train_model(epochs, print_every=100):
    model.cuda(1)
    model.train()

    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
       
        for i, batch in enumerate(train_iter):            
            src = batch.English.transpose(0,1)
            trg = batch.German.transpose(0,1)            
            
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            if torch.cuda.is_available():
                src = src.cuda(1)
                trg_input = trg_input.cuda(1)
            
            #print(src.is_cuda)    
            #print(trg_input.is_cuda)
            #print(input_msk.is_cuda)
            #print(target_msk.is_cuda)
            
            src_mask, trg_mask = create_masks(src, trg_input)
            #print(src_mask)
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)),targets.cuda(1), ignore_index=target_pad)            
            loss.backward()
            optim.step()
            
            #print(loss.data)
            total_loss += loss.data
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60, epoch + 1, i + 1, loss_avg, time.time() - temp,print_every))
                total_loss = 0
                temp = time.time()

In [59]:
train_model(10, print_every=10)

time = 0m, epoch 1, iter = 10, loss = 2.988, 0s per 10 iters
time = 0m, epoch 1, iter = 20, loss = 3.398, 0s per 10 iters
time = 0m, epoch 1, iter = 30, loss = 3.079, 0s per 10 iters
time = 0m, epoch 1, iter = 40, loss = 3.155, 0s per 10 iters
time = 0m, epoch 1, iter = 50, loss = 3.432, 0s per 10 iters
time = 0m, epoch 1, iter = 60, loss = 3.361, 0s per 10 iters
time = 0m, epoch 1, iter = 70, loss = 3.611, 0s per 10 iters
time = 0m, epoch 1, iter = 80, loss = 3.466, 0s per 10 iters
time = 0m, epoch 1, iter = 90, loss = 3.248, 0s per 10 iters
time = 0m, epoch 1, iter = 100, loss = 3.382, 0s per 10 iters
time = 0m, epoch 1, iter = 110, loss = 3.092, 0s per 10 iters
time = 0m, epoch 1, iter = 120, loss = 2.865, 0s per 10 iters
time = 0m, epoch 1, iter = 130, loss = 3.169, 0s per 10 iters
time = 0m, epoch 1, iter = 140, loss = 2.950, 0s per 10 iters
time = 0m, epoch 1, iter = 150, loss = 3.316, 0s per 10 iters
time = 0m, epoch 1, iter = 160, loss = 3.267, 0s per 10 iters
time = 0m, epoch 

time = 0m, epoch 1, iter = 1330, loss = 2.819, 0s per 10 iters
time = 0m, epoch 1, iter = 1340, loss = 3.339, 0s per 10 iters
time = 0m, epoch 1, iter = 1350, loss = 3.491, 0s per 10 iters
time = 0m, epoch 1, iter = 1360, loss = 2.744, 0s per 10 iters
time = 0m, epoch 1, iter = 1370, loss = 3.376, 0s per 10 iters
time = 0m, epoch 1, iter = 1380, loss = 3.267, 0s per 10 iters
time = 0m, epoch 1, iter = 1390, loss = 3.472, 0s per 10 iters
time = 0m, epoch 1, iter = 1400, loss = 3.178, 0s per 10 iters
time = 0m, epoch 1, iter = 1410, loss = 3.045, 0s per 10 iters
time = 0m, epoch 1, iter = 1420, loss = 3.073, 0s per 10 iters
time = 0m, epoch 1, iter = 1430, loss = 3.279, 0s per 10 iters
time = 0m, epoch 1, iter = 1440, loss = 3.136, 0s per 10 iters
time = 0m, epoch 1, iter = 1450, loss = 2.879, 0s per 10 iters
time = 0m, epoch 1, iter = 1460, loss = 3.408, 0s per 10 iters
time = 0m, epoch 1, iter = 1470, loss = 4.105, 0s per 10 iters
time = 0m, epoch 1, iter = 1480, loss = 3.428, 0s per 1

time = 1m, epoch 1, iter = 2640, loss = 3.238, 0s per 10 iters
time = 1m, epoch 1, iter = 2650, loss = 3.346, 0s per 10 iters
time = 1m, epoch 1, iter = 2660, loss = 3.148, 0s per 10 iters
time = 1m, epoch 1, iter = 2670, loss = 2.813, 0s per 10 iters
time = 1m, epoch 1, iter = 2680, loss = 3.077, 0s per 10 iters
time = 1m, epoch 1, iter = 2690, loss = 3.420, 0s per 10 iters
time = 1m, epoch 1, iter = 2700, loss = 2.993, 0s per 10 iters
time = 1m, epoch 1, iter = 2710, loss = 3.447, 0s per 10 iters
time = 1m, epoch 1, iter = 2720, loss = 3.296, 0s per 10 iters
time = 1m, epoch 1, iter = 2730, loss = 2.959, 0s per 10 iters
time = 1m, epoch 1, iter = 2740, loss = 3.256, 0s per 10 iters
time = 1m, epoch 1, iter = 2750, loss = 2.750, 0s per 10 iters
time = 1m, epoch 1, iter = 2760, loss = 3.418, 0s per 10 iters
time = 1m, epoch 1, iter = 2770, loss = 3.210, 0s per 10 iters
time = 1m, epoch 1, iter = 2780, loss = 3.149, 0s per 10 iters
time = 1m, epoch 1, iter = 2790, loss = 3.789, 0s per 1

time = 2m, epoch 1, iter = 3950, loss = 2.600, 0s per 10 iters
time = 2m, epoch 1, iter = 3960, loss = 3.314, 0s per 10 iters
time = 2m, epoch 1, iter = 3970, loss = 2.733, 0s per 10 iters
time = 2m, epoch 1, iter = 3980, loss = 3.447, 0s per 10 iters
time = 2m, epoch 1, iter = 3990, loss = 3.359, 0s per 10 iters
time = 2m, epoch 1, iter = 4000, loss = 3.186, 0s per 10 iters
time = 2m, epoch 1, iter = 4010, loss = 3.634, 0s per 10 iters
time = 2m, epoch 1, iter = 4020, loss = 3.199, 0s per 10 iters
time = 2m, epoch 1, iter = 4030, loss = 3.058, 0s per 10 iters
time = 2m, epoch 1, iter = 4040, loss = 3.609, 0s per 10 iters
time = 2m, epoch 1, iter = 4050, loss = 3.303, 0s per 10 iters
time = 2m, epoch 1, iter = 4060, loss = 3.489, 0s per 10 iters
time = 2m, epoch 1, iter = 4070, loss = 3.143, 0s per 10 iters
time = 2m, epoch 1, iter = 4080, loss = 3.245, 0s per 10 iters
time = 2m, epoch 1, iter = 4090, loss = 3.141, 0s per 10 iters
time = 2m, epoch 1, iter = 4100, loss = 3.477, 0s per 1

time = 3m, epoch 1, iter = 5260, loss = 3.522, 0s per 10 iters
time = 3m, epoch 1, iter = 5270, loss = 3.320, 0s per 10 iters
time = 3m, epoch 1, iter = 5280, loss = 3.102, 0s per 10 iters
time = 3m, epoch 1, iter = 5290, loss = 2.977, 0s per 10 iters
time = 3m, epoch 1, iter = 5300, loss = 2.934, 0s per 10 iters
time = 3m, epoch 1, iter = 5310, loss = 3.031, 0s per 10 iters
time = 3m, epoch 1, iter = 5320, loss = 3.248, 0s per 10 iters
time = 3m, epoch 1, iter = 5330, loss = 3.067, 0s per 10 iters
time = 3m, epoch 1, iter = 5340, loss = 3.115, 0s per 10 iters
time = 3m, epoch 1, iter = 5350, loss = 3.552, 0s per 10 iters
time = 3m, epoch 1, iter = 5360, loss = 3.258, 0s per 10 iters
time = 3m, epoch 1, iter = 5370, loss = 3.181, 0s per 10 iters
time = 3m, epoch 1, iter = 5380, loss = 3.728, 0s per 10 iters
time = 3m, epoch 1, iter = 5390, loss = 3.216, 0s per 10 iters
time = 3m, epoch 1, iter = 5400, loss = 3.277, 0s per 10 iters
time = 3m, epoch 1, iter = 5410, loss = 3.507, 0s per 1

...

time = 146m, epoch 10, iter = 15730, loss = 3.182, 0s per 10 iters
time = 146m, epoch 10, iter = 15740, loss = 3.901, 0s per 10 iters
time = 146m, epoch 10, iter = 15750, loss = 2.983, 0s per 10 iters
time = 146m, epoch 10, iter = 15760, loss = 3.279, 0s per 10 iters
time = 146m, epoch 10, iter = 15770, loss = 3.164, 0s per 10 iters
time = 146m, epoch 10, iter = 15780, loss = 3.025, 0s per 10 iters
time = 146m, epoch 10, iter = 15790, loss = 3.263, 0s per 10 iters
time = 146m, epoch 10, iter = 15800, loss = 2.845, 0s per 10 iters
time = 146m, epoch 10, iter = 15810, loss = 3.015, 0s per 10 iters
time = 146m, epoch 10, iter = 15820, loss = 3.579, 0s per 10 iters
time = 146m, epoch 10, iter = 15830, loss = 3.154, 0s per 10 iters
time = 146m, epoch 10, iter = 15840, loss = 3.152, 0s per 10 iters
time = 146m, epoch 10, iter = 15850, loss = 3.410, 0s per 10 iters
time = 146m, epoch 10, iter = 15860, loss = 3.483, 0s per 10 iters
time = 146m, epoch 10, iter = 15870, loss = 3.123, 0s per 10 i

time = 146m, epoch 10, iter = 16960, loss = 2.815, 0s per 10 iters
time = 146m, epoch 10, iter = 16970, loss = 3.098, 0s per 10 iters
time = 146m, epoch 10, iter = 16980, loss = 3.010, 0s per 10 iters
time = 146m, epoch 10, iter = 16990, loss = 2.485, 0s per 10 iters
time = 146m, epoch 10, iter = 17000, loss = 2.732, 0s per 10 iters
time = 146m, epoch 10, iter = 17010, loss = 3.075, 0s per 10 iters
time = 147m, epoch 10, iter = 17020, loss = 2.875, 0s per 10 iters
time = 147m, epoch 10, iter = 17030, loss = 3.742, 0s per 10 iters
time = 147m, epoch 10, iter = 17040, loss = 3.387, 0s per 10 iters
time = 147m, epoch 10, iter = 17050, loss = 3.013, 0s per 10 iters
time = 147m, epoch 10, iter = 17060, loss = 2.697, 0s per 10 iters
time = 147m, epoch 10, iter = 17070, loss = 2.254, 0s per 10 iters
time = 147m, epoch 10, iter = 17080, loss = 3.768, 0s per 10 iters
time = 147m, epoch 10, iter = 17090, loss = 2.767, 0s per 10 iters
time = 147m, epoch 10, iter = 17100, loss = 2.747, 0s per 10 i

time = 147m, epoch 10, iter = 18190, loss = 2.921, 0s per 10 iters
time = 147m, epoch 10, iter = 18200, loss = 3.306, 0s per 10 iters
time = 147m, epoch 10, iter = 18210, loss = 3.172, 0s per 10 iters
time = 147m, epoch 10, iter = 18220, loss = 2.995, 0s per 10 iters
time = 147m, epoch 10, iter = 18230, loss = 2.885, 0s per 10 iters
time = 147m, epoch 10, iter = 18240, loss = 2.869, 0s per 10 iters
time = 147m, epoch 10, iter = 18250, loss = 3.388, 0s per 10 iters
time = 147m, epoch 10, iter = 18260, loss = 3.252, 0s per 10 iters
time = 147m, epoch 10, iter = 18270, loss = 3.148, 0s per 10 iters
time = 147m, epoch 10, iter = 18280, loss = 3.575, 0s per 10 iters
time = 147m, epoch 10, iter = 18290, loss = 2.947, 0s per 10 iters
time = 147m, epoch 10, iter = 18300, loss = 3.361, 0s per 10 iters
time = 147m, epoch 10, iter = 18310, loss = 2.811, 0s per 10 iters
time = 147m, epoch 10, iter = 18320, loss = 3.103, 0s per 10 iters
time = 147m, epoch 10, iter = 18330, loss = 3.806, 0s per 10 i

time = 148m, epoch 10, iter = 19420, loss = 3.497, 0s per 10 iters
time = 148m, epoch 10, iter = 19430, loss = 3.429, 0s per 10 iters
time = 148m, epoch 10, iter = 19440, loss = 3.063, 0s per 10 iters
time = 148m, epoch 10, iter = 19450, loss = 3.332, 0s per 10 iters
time = 148m, epoch 10, iter = 19460, loss = 3.452, 0s per 10 iters
time = 148m, epoch 10, iter = 19470, loss = 3.684, 0s per 10 iters
time = 148m, epoch 10, iter = 19480, loss = 2.925, 0s per 10 iters
time = 148m, epoch 10, iter = 19490, loss = 2.607, 0s per 10 iters
time = 148m, epoch 10, iter = 19500, loss = 3.465, 0s per 10 iters
time = 148m, epoch 10, iter = 19510, loss = 3.216, 0s per 10 iters
time = 148m, epoch 10, iter = 19520, loss = 2.923, 0s per 10 iters
time = 148m, epoch 10, iter = 19530, loss = 3.109, 0s per 10 iters
time = 148m, epoch 10, iter = 19540, loss = 3.095, 0s per 10 iters
time = 148m, epoch 10, iter = 19550, loss = 3.465, 0s per 10 iters
time = 148m, epoch 10, iter = 19560, loss = 3.563, 0s per 10 i

time = 149m, epoch 10, iter = 20650, loss = 3.340, 0s per 10 iters
time = 149m, epoch 10, iter = 20660, loss = 3.482, 0s per 10 iters
time = 149m, epoch 10, iter = 20670, loss = 3.248, 0s per 10 iters
time = 149m, epoch 10, iter = 20680, loss = 3.227, 0s per 10 iters
time = 149m, epoch 10, iter = 20690, loss = 3.271, 0s per 10 iters
time = 149m, epoch 10, iter = 20700, loss = 3.667, 0s per 10 iters
time = 149m, epoch 10, iter = 20710, loss = 2.925, 0s per 10 iters
time = 149m, epoch 10, iter = 20720, loss = 3.416, 0s per 10 iters
time = 149m, epoch 10, iter = 20730, loss = 2.653, 0s per 10 iters
time = 149m, epoch 10, iter = 20740, loss = 2.969, 0s per 10 iters
time = 149m, epoch 10, iter = 20750, loss = 3.481, 0s per 10 iters


In [61]:
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            }, "/media/data6TB/deepak/NalinIdea/deu-eng/real_de_en_20.pt")

In [68]:
model

Transformer(
  (encoder): Encoder(
    (embed): Embedder(
      (embed): Embedding(16044, 80)
    )
    (pe): PositionalEncoder()
    (layers): ModuleList(
      (0): EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (attn): MultiHeadAttention(
          (q_linear): Linear(in_features=80, out_features=80, bias=True)
          (v_linear): Linear(in_features=80, out_features=80, bias=True)
          (k_linear): Linear(in_features=80, out_features=80, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (out): Linear(in_features=80, out_features=80, bias=True)
        )
        (ff): FeedForward(
          (linear_1): Linear(in_features=80, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear_2): Linear(in_features=2048, out_features=80, bias=True)
        )
        (dropout_1): Dropout(p=0.1, inplace=False)
        (dropout_2): Dropout(p=0.1, inplace=False)
      )
      (1): EncoderLayer(
        (nor

In [62]:
def translate(model, src, max_len = 90, custom_sentence=False):
    
    model.eval()
    if custom_sentence == True:
        src = tokenize_de(src)
        sentence= Variable(torch.LongTensor([[EN_TEXT.vocab.stoi[tok] for tok in src]])).cuda(1)
    src_mask = (sentence != input_pad).unsqueeze(-2)
    e_outputs = model.encoder(sentence, src_mask)
    
    outputs = torch.zeros(max_len).type_as(sentence.data)
    outputs[0] = torch.LongTensor([DE_TEXT.vocab.stoi['<sos>']])
    for i in range(1, max_len):
        trg_mask = np.triu(np.ones((1, i, i)), k=1).astype('uint8')
        trg_mask = Variable(torch.from_numpy(trg_mask) == 0).cuda(1) 
        out = model.out(model.decoder(outputs[:i].unsqueeze(0), e_outputs, src_mask, trg_mask))
        out = F.softmax(out, dim=-1)
        val, ix = out[:, -1].data.topk(1)
        
        outputs[i] = ix[0][0]
        if ix[0][0] == DE_TEXT.vocab.stoi['<eos>']:
            break
    return ' '.join([DE_TEXT.vocab.itos[ix] for ix in outputs[:i]])

In [34]:
import pandas as pd

validation_data=pd.read_csv('/media/data6TB/deepak/NalinIdea/deu-eng/new_val.csv')

In [42]:
validation_data['English'][2]

'Tom usually seems to agree with Mary.'

In [43]:
translate(model,src=validation_data['English'][2],custom_sentence=True)

'<sos> Tom scheint Maria gewöhnlich mit Maria zu .'

In [63]:
translated_validation=[]
for i in range(len(validation_data)):
    translated_validation.append(translate(model,src=validation_data['English'][i],custom_sentence=True))

In [56]:
#raw_data = {'English': translated_validation}
#df = pd.DataFrame(raw_data, columns=["English"])
#df.to_csv("translated_de_en_validation.csv", index=False)

In [57]:
#test = pd.read_csv('translated_de_en_validation.csv')

In [64]:
preds=translated_validation
refs=[]
for i in range(len(validation_data)):
    refs.append(validation_data['German'][i])

In [65]:
import sacrebleu
bleu= sacrebleu.corpus_bleu(preds,[refs])
print(bleu.score)

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with '--force'.


10.797245959382225


In [40]:
list_refs=[]
hypo=[]
for i in range(len(validation_data)):
    list_refs.append([refs[0][i].split()])
    hypo.append(preds[i].split())

In [41]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu

smoothie=SmoothingFunction().method4
corpus_bleu(list_refs, hypo,  smoothing_function=smoothie)

0.005865125935848249

In [51]:
translated_validationC=[]
for i in range(len(validation_data)):
    translated_validationC.append(translate(modelC,src=validation_data['German'][i],custom_sentence=True))

In [52]:
refs=[translated_validationC]
preds=[]
for i in range(len(validation_data)):
    preds.append(validation_data['English'][i])

In [53]:
import sacrebleu
bleu= sacrebleu.corpus_bleu(preds,refs)
print(bleu.score)

13.084072650038953


In [54]:
list_refs=[]
hypo=[]
for i in range(len(validation_data)):
    list_refs.append([refs[0][i].split()])
    hypo.append(preds[i].split())

In [55]:
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu

smoothie=SmoothingFunction().method4
corpus_bleu(list_refs, hypo,  smoothing_function=smoothie)

0.10544157497649873